# Analysing Political Executives with AI

## Using AutoGluon to processing the data


```shell
# Link for how to install autogluon on Windows system
# https://auto.gluon.ai/stable/index.html
# For other operation systems, please also check this link for latest news. 

# test automl model
# install autogluon
!pip3 install -U pip
!pip3 install -U setuptools wheel

# CPU version of pytorch has smaller footprint - see installation instructions in
# pytorch documentation - https://pytorch.org/get-started/locally/
!pip3 install torch==1.10.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

!pip3 install autogluon
```

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split #splitting the dataset
import numpy as np

# Showing all the details of results
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

In [ ]:
# Import the data file
train_data = TabularDataset('./../Data/data_AutoGluon.csv')

In [ ]:
type(train_data)

train_data.columns

train_data.head()

train_data.describe()

In [ ]:
# delete other irrelevant columns
del train_data['growth_rate_normgroup']
# del train_data['growth_rate_avggrp']
del train_data['growth_rate']
train_data.columns

# change label into growth_rate
# id, label = 'leader', 'growth_rate'

# change label into growth_rate_normgroup
# id, label = 'leader', 'growth_rate_normgroup'

# change label into growth_rate group
id, label = 'leader', 'growth_rate_avggrp'
df_train = train_data

In [ ]:
df_train.dtypes

In [ ]:
#split train data and test data
# df_train, df_test=train_test_split(train_data, test_size=0.33, random_state=1)
# df_train.shape, df_test.shape